### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
query='love'
filter_query = 'section_name:("Movies") AND type_of_material:("Review") AND headline:("love")'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url=f"{url}api-key={nyt_api_key}&q={query}&fq={filter_query}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&sort={sort}"

In [4]:
#Check
page_url=query_url
articles = requests.get(query_url).json()
articles_list = articles["response"]["docs"]
print(json.dumps(articles_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
    # create query with a page number
    # API results show 10 articles at a time

for page in range(0,20):
    page_url=f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    articles = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        articles = articles["response"]["docs"]
        # loop through the reviews["response"]["docs"] and append each review to the list
        for article in articles:
            reviews_list.append(article)
            content=len(reviews_list)
        # Print the page that was just retrieved
        print(f"the page number retrieved is:{page}") 
        
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"There is an error retrieving articles on page :{page}") 
        break
#print(json.dumps(reviews_list, indent=4))

the page number retrieved is:0
the page number retrieved is:1
the page number retrieved is:2
the page number retrieved is:3
the page number retrieved is:4
the page number retrieved is:5
the page number retrieved is:6
the page number retrieved is:7
the page number retrieved is:8
the page number retrieved is:9
the page number retrieved is:10
the page number retrieved is:11
the page number retrieved is:12
the page number retrieved is:13
the page number retrieved is:14
the page number retrieved is:15
the page number retrieved is:16
the page number retrieved is:17
the page number retrieved is:18
the page number retrieved is:19


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range (0,5):
    print(json.dumps(reviews_list[i], indent=4))

{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
' I tried the suggested lambda function and an alternative and my procees gave a cleaner result for the title'
'I have inlcuded the working lambda function in the comment below in case you would like to check both'

reviews_df = pd.json_normalize(reviews_list)
reviews_df['headline.main2']=reviews_df['headline.main'].str.replace("’s","++s")
reviews_df['title']=reviews_df['headline.main2'].str.extract(r"\u2018(.*?)\u2019", expand=True)
#reviews_df['title']= reviews_df['headline.main2']. apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df['title']=reviews_df['title'].str.replace("++s","'s")
reviews_df['title']=reviews_df['title'].str.rstrip(",")
reviews_df[['headline.main','headline.main2','title']]. head(30)
reviews_df=reviews_df.drop('headline.main2', axis =1)
reviews_df.head(10)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What's Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist's Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People's Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times

In [9]:
# Extract 'name' and 'value' from items in "keywords" column
reviews_df2=reviews_df
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}:{item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df2['keywords']= reviews_df2['keywords'].apply(extract_keywords)
reviews_df2.head(10)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject:Movies;creative_works:The Attachment D...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject:Movies;persons:Kapur, Shekhar;persons:...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What's Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject:Movies;creative_works:You Can Live For...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject:Movies;creative_works:A Tourist's Guid...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist's Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject:Movies;persons:Zlotowski, Rebecca;crea...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People's Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject:Movies;persons:Bracey, Luke (1989- );p...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject:Documentary Films and Programs;creativ...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject:Movies;creative_works:A Thousand and O...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject:Movies;creative_works:Your Place or Mi...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject:Doc

In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = reviews_df['title'].tolist()
title_list

['The Attachment Diaries',
 "What's Love Got to Do With It?",
 'You Can Live Forever',
 "A Tourist's Guide to Love",
 "Other People's Children",
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 "Lady Chatterley's Lover",
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journa

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
url2='https://api.themoviedb.org/3/movie/'
tmdb_key_string1 = "&api_key=" + tmdb_api_key
tmdb_key_string2 = "?api_key=" + tmdb_api_key

In [12]:
  # Make a request for a the full movie details
#movie_url = f"{url2}{movie_id}{tmdb_key_string2}"
  #movie_details = requests.get(movie_url).json()
#print(json.dumps( movie_details, indent=4))

In [13]:
# Create an empty list to store the results
movie_list=[]
# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter=1
# Loop through the titles
for title in title_list:
    # Add 1 to the request counter
    request_counter=request_counter+1
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print(" The application is sleeping")
    else:
         # Perform a "GET" request for The Movie Database
        query_url= f"{url}{title}{tmdb_key_string1}"
        articles = requests.get(query_url).json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        try:
         # Get movie id
            movie_id = articles['results'][0]['id']

        # Make a request for a the full movie details
            movie_url = f"{url2}{movie_id}{tmdb_key_string2}"

        # Execute "GET" request with url
            movie_details = requests.get(movie_url).json()
        
        # Extract the genre names into a list
            genres=[genre['name'] for genre in movie_details['genres']]

        # Extract the spoken_languages' English name into a list
            lang=[lang['english_name'] for lang in movie_details['spoken_languages']]

        # Extract the production_countries' name into a list

            prod_country=[country['name'] for country in movie_details['production_countries']]
        # Add the relevant data to a dictionary and
            movie_key_details ={
                            'title': movie_details['title'],
                            'original_title' : movie_details['original_title'],
                            'budget': movie_details['budget'],
                            'Genre': genres,
                            'language': movie_details['original_language'],
                            'spoken_language': lang,
                            'homepage':movie_details['homepage'],
                            'overview':movie_details['overview'],
                            'popularity':movie_details['popularity'],
                            'runtime':movie_details['runtime'],
                            'revenue':movie_details['revenue'],
                            'release_date':movie_details['release_date'],
                            'vote_average':movie_details['vote_average'],
                            'vote_count':movie_details['vote_count'],
                            'production_countries': prod_country
                            }
        # append it to the tmdb_movies_list list

            movie_list.append(movie_key_details)
        # Print out the title that was found
            print(f" Found Movie {title}")
        except:
            print(f" Not Found Movie {title} ")

 Found Movie The Attachment Diaries
 Found Movie What's Love Got to Do With It?
 Found Movie You Can Live Forever
 Found Movie A Tourist's Guide to Love
 Found Movie Other People's Children
 Found Movie One True Loves
 Found Movie The Lost Weekend: A Love Story
 Found Movie A Thousand and One
 Found Movie Your Place or Mine
 Found Movie Love in the Time of Fentanyl
 Found Movie Pamela, a Love Story
 Found Movie In From the Side
 Found Movie After Love
 Found Movie Alcarràs
 Found Movie Nelly & Nadine
 Found Movie Lady Chatterley's Lover
 Found Movie The Sound of Christmas
 Found Movie The Inspection
 Found Movie Bones and All
 Found Movie My Policeman
 Found Movie About Fate
 Found Movie Waiting for Bojangles
 Found Movie I Love My Dad
 Found Movie A Love Song
 Found Movie Alone Together
 Found Movie Art of Love
 Found Movie The Wheel
 Found Movie Thor: Love and Thunder
 Found Movie Both Sides of the Blade
 Found Movie Fire of Love
 Found Movie Love & Gelato
 Found Movie Stay Prayed Up

In [14]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range (0,5):
    print(json.dumps(movie_list[i], indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "Genre": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "language": "es",
    "spoken_language": [
        "Spanish"
    ],
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 7.021,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "What's Love Got to Do with It?",
    "original_title": "What's Love Got to Do with It?",
    "bu

In [15]:
# Convert the results to a DataFrame
tmdb_df= pd.DataFrame(movie_list)
tmdb_df.head(10)

,title,original_title,budget,Genre,language,spoken_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",7.021,102,0,2021-10-07,3.000,4,[Argentina]
1,What's Love Got to Do with It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Portuguese, Urdu]",,Two childhood friends now in their thirties mu...,38.657,109,10898395,2023-01-26,6.077,181,"[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",22.548,96,15055,2023-03-24,6.564,39,"[Canada, United States of America]"
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.844,96,0,2023-04-21,6.311,167,[United States of America]
4,Other People's Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",14.299,104,84178,2022-09-21,6.820,192,[France]
5,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.688,100,37820,2023-04-07,6.500,79,"[Czech Republic, United States of America]"
6,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,20.433,95,77145,2023-04-13,6.000,2,[United States of America]
7,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Spanish, Portuguese]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,14.223,116,3395595,2023-03-31,6.872,90,[United States of America]
8,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",en,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,33.866,109,0,2023-02-10,6.300,729,[United States of America]
9,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,1.055,85,0,2023-02-03,0.000,0,"[Canada, United States of America]"


In [16]:
# pre check prior to merge to check for row count
reviews_df2.info() # 200 entries 
tmdb_df.info() #198 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   web_url                  200 non-null    object
 1   snippet                  200 non-null    object
 2   source                   200 non-null    object
 3   keywords                 200 non-null    object
 4   pub_date                 200 non-null    object
 5   word_count               200 non-null    int64 
 6   headline.main            200 non-null    object
 7   headline.kicker          47 non-null     object
 8   headline.content_kicker  0 non-null      object
 9   headline.print_headline  199 non-null    object
 10  headline.name            0 non-null      object
 11  headline.seo             0 non-null      object
 12  headline.sub             0 non-null      object
 13  byline.original          200 non-null    object
 14  byline.person            200 non-null    o

### Merge and Clean the Data for Export

In [17]:
# Merge the New York Times reviews and TMDB DataFrames on title
nytimes_tmdb_merged_df =pd.merge(tmdb_df, reviews_df2, how='left',on='title')
nytimes_tmdb_merged_df.info()
nytimes_tmdb_merged_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    199 non-null    object 
 1   original_title           199 non-null    object 
 2   budget                   199 non-null    int64  
 3   Genre                    199 non-null    object 
 4   language                 199 non-null    object 
 5   spoken_language          199 non-null    object 
 6   homepage                 199 non-null    object 
 7   overview                 199 non-null    object 
 8   popularity               199 non-null    float64
 9   runtime                  199 non-null    int64  
 10  revenue                  199 non-null    int64  
 11  release_date             199 non-null    object 
 12  vote_average             199 non-null    float64
 13  vote_count               199 non-null    int64  
 14  production_countries     1

,title,original_title,budget,Genre,language,spoken_language,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",7.021,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What's Love Got to Do with It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Portuguese, Urdu]",,Two childhood friends now in their thirties mu...,38.657,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
columns_to_fix =['Genre','spoken_language','production_countries']
new_nytimes_tmdb_merged_df = nytimes_tmdb_merged_df
# Create a list of characters to remove

characters_to_remove=['[',']',"'"]
len(characters_to_remove)

3

In [19]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix =['Genre','spoken_language','production_countries']
new_nytimes_tmdb_merged_df = nytimes_tmdb_merged_df
# Create a list of characters to remove
characters_to_remove=['[',']',"'"]
len(characters_to_remove)

# Loop through the list of columns to fix
for col in columns_to_fix:
    for char in range(0,(len(characters_to_remove))):
        # Convert the column to type 'str'
        new_nytimes_tmdb_merged_df[col] = nytimes_tmdb_merged_df[col].astype(str)
        # Loop through characters to remove
        new_nytimes_tmdb_merged_df[col] =  nytimes_tmdb_merged_df[col].str.replace(characters_to_remove[char],'')

# Display the fixed DataFrame
new_nytimes_tmdb_merged_df.head(10)

,title,original_title,budget,Genre,language,spoken_language,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",7.021,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What's Love Got to Do with It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,38.657,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",22.548,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.844,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People's Children,Les Enfants des autres,0,"Drama, Comedy",fr,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",14.299,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,One True Loves,One True Loves,0,"Romance, Comedy, Drama",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,14.688,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,20.433,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,14.223,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,Your Place or Mine,Your Place or Mine,0,"Romance, Comedy",en,English,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,33.866,109,...,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,Documentary,en,English,,An intimate portrait of a community fighting t...,1.055,85,...,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None


In [20]:
# Drop "byline.person" column
new_nytimes_tmdb_merged_df = new_nytimes_tmdb_merged_df.drop('byline.person', axis=1)
new_nytimes_tmdb_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    199 non-null    object 
 1   original_title           199 non-null    object 
 2   budget                   199 non-null    int64  
 3   Genre                    199 non-null    object 
 4   language                 199 non-null    object 
 5   spoken_language          199 non-null    object 
 6   homepage                 199 non-null    object 
 7   overview                 199 non-null    object 
 8   popularity               199 non-null    float64
 9   runtime                  199 non-null    int64  
 10  revenue                  199 non-null    int64  
 11  release_date             199 non-null    object 
 12  vote_average             199 non-null    float64
 13  vote_count               199 non-null    int64  
 14  production_countries     1

In [21]:
# Delete duplicate rows and reset index
new_nytimes_tmdb_merged_df2=new_nytimes_tmdb_merged_df[~new_nytimes_tmdb_merged_df.astype(str).duplicated()]
new_nytimes_tmdb_merged_df2=new_nytimes_tmdb_merged_df2.reset_index(drop=True)

In [22]:
new_nytimes_tmdb_merged_df2.info()
new_nytimes_tmdb_merged_df2.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    191 non-null    object 
 1   original_title           191 non-null    object 
 2   budget                   191 non-null    int64  
 3   Genre                    191 non-null    object 
 4   language                 191 non-null    object 
 5   spoken_language          191 non-null    object 
 6   homepage                 191 non-null    object 
 7   overview                 191 non-null    object 
 8   popularity               191 non-null    float64
 9   runtime                  191 non-null    int64  
 10  revenue                  191 non-null    int64  
 11  release_date             191 non-null    object 
 12  vote_average             191 non-null    float64
 13  vote_count               191 non-null    int64  
 14  production_countries     1

,title,original_title,budget,Genre,language,spoken_language,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",7.021,102,...,295.0,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What's Love Got to Do with It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,38.657,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",22.548,96,...,294.0,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",25.844,96,...,276.0,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People's Children,Les Enfants des autres,0,"Drama, Comedy",fr,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",14.299,104,...,801.0,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
5,One True Loves,One True Loves,0,"Romance, Comedy, Drama",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,14.688,100,...,320.0,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
6,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,20.433,95,...,327.0,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
7,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,14.223,116,...,971.0,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
8,Your Place or Mine,Your Place or Mine,0,"Romance, Comedy",en,English,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,33.866,109,...,569.0,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,None
9,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,Documentary,en,English,,An intimate portrait of a community fighting t...,1.055,85,...,306.0,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,None


In [23]:
# Export data to CSV without the index
new_nytimes_tmdb_merged_df2.to_csv('/Users/Uday/projects/data-sourcing-challenge/nytimes_tmdb_collected_data.csv', index=False)